In [1]:
from dotenv import load_dotenv
from jira import JIRA

In [2]:
load_dotenv()
domain = os.getenv('JIRA_DOMAIN')
username = os.getenv('JIRA_USERNAME')
api_token = os.getenv('JIRA_API_TOKEN')
project_key = os.getenv('JIRA_PROJECT_KEY')
jira_url = f'https://{domain}.atlassian.net'
jira_options = {'server': jira_url}
jira = JIRA(options=jira_options, basic_auth=(username, api_token))

# project id van Datateam MOSS+
project_key = 'DSO'

In [3]:
# get all members in project with project_key
project = jira.project(project_key)

In [4]:
[user for user in jira.search_assignable_users_for_projects('', project_key) if user.active]

[<JIRA User: displayName='Alexei  Ivantchik', accountId='615ad6cf7a6be40071e5f138'>,
 <JIRA User: displayName='Bastiaan  Nota', accountId='712020:87693d57-69a7-4372-b90c-1fd64f1dd867'>,
 <JIRA User: displayName='Bert  Hulst', accountId='70121:2a3f903e-3678-4736-a74f-cfd3d720097d'>,
 <JIRA User: displayName='Cees Klomp', accountId='5aead35cba41192e23d883b4'>,
 <JIRA User: displayName='Claire de Visscher, de', accountId='5ff6d7019edf280075979253'>,
 <JIRA User: displayName='Cloud Support Prepend', accountId='6332eecb2eaaa5dcfa14887b'>,
 <JIRA User: displayName='Dave Kleihorst', accountId='5ca33c8f8b98067a17152d74'>,
 <JIRA User: displayName='Edwin Stol [Prepend]', accountId='557058:19e1f5f5-1c06-4a6d-8da1-62c0d58f1551'>,
 <JIRA User: displayName='Fried  Scholvinck', accountId='712020:0860a03b-61fc-4e5d-8dc8-b21c141c926f'>,
 <JIRA User: displayName='Hugo  Rensink', accountId='62e7bfc848e310e67294f188'>,
 <JIRA User: displayName='Jamal  Zerri', accountId='5ca1cb2b7a6bf9261a41dce6'>,
 <JIRA

In [5]:
fields = jira.fields()

# Find the story points field
for f in fields:
    if 'story points' in f['name'].lower():  # Adjust the search term if necessary
        print("Field Name:", f['name'])
        print("Field ID:", f['id'])
        break

Field Name: Story Points
Field ID: customfield_10004


In [6]:
f

{'id': 'customfield_10004',
 'key': 'customfield_10004',
 'name': 'Story Points',
 'untranslatedName': 'Story Points',
 'custom': True,
 'orderable': True,
 'navigable': True,
 'searchable': True,
 'clauseNames': ['cf[10004]', 'Story Points', 'Story Points[Number]'],
 'schema': {'type': 'number',
  'custom': 'com.atlassian.jira.plugin.system.customfieldtypes:float',
  'customId': 10004}}

In [7]:
def get_team_members(project):
    ''' Get all team members in a project and return a dictionary with key = name and value = accountId 
    '''
    team_members = set()
    
    # Fetch issues in the project and add assignees to the team_members set
    for issue in jira.search_issues(f'project={project.key}', maxResults=1000):
        if issue.fields.assignee:
            team_members.add(issue.fields.assignee)

    # get all team members in dictionary with key = name and value = accountId, also get the role of the team member
    team_members_dict = {}
    for team_member in team_members:
        team_members_dict[team_member.displayName.split(' ')[0]] = team_member.accountId    

    return team_members_dict

In [8]:
team = get_team_members(project)
team

{'Remco': '557058:cde08b1e-8d63-4912-b0f3-e1d64b1dc21b',
 'Jamal': '5ca1cb2b7a6bf9261a41dce6',
 'Yirong': '712020:3160fe4a-e580-42d3-9dda-7a0550a89ce3',
 'Alexei': '615ad6cf7a6be40071e5f138',
 'Moethoeli': '70121:3c8c8352-9d70-423f-aa88-5111afaf654d',
 'Nick': '712020:d37a6bae-9360-486a-9f6b-f7aaa9ec380b',
 'Koen': '712020:bc411c66-09c3-4346-83f6-e7d361cd026b',
 'Peter': '615ad8ea07ac3c0068fede31',
 'Joost': '62866190d9ddcc006e9eac96',
 'Fried': '712020:0860a03b-61fc-4e5d-8dc8-b21c141c926f',
 'Robbin': '712020:3df5e74d-a3ad-4183-8620-0c195c971498',
 'Khalid': '6405b6c715d668edd8ef17c3',
 'Miguel': '63d93478c3eb74ad8e952b7d',
 'Bert': '70121:2a3f903e-3678-4736-a74f-cfd3d720097d'}

In [9]:
assigned_roles = {
    'business_analist': 'Fried',
    'informatie_analist': 'Fried',
    'data_engineer': 'Fried',
    'bi_specialist': 'Fried',
}

project_input = {
    'name': 'Test Dashboard',
    'sub_domain': 'S&B', # kies 1 van ['MV', 'ON', 'SUB', 'S&B']
    'sub_project': 'TD1',
    'description': 'Dit is een test dashboard.',
    'year': 2024,
    'quarter': 1,
    'team_members': assigned_roles
}

# assert that all names are in the team keys
for role in assigned_roles.values():
    assert role in team.keys()

In [10]:
epic_base_title = f"[Epic {project_input['sub_domain']}_{project_input['sub_project']}_{project_input['year']}]"
feature_base_title = f"[Feature {project_input['sub_domain']}_{project_input['sub_project']}_{project_input['year']}]"
story_base_title = f"[{project_input['sub_domain']}_{project_input['sub_project']}_{project_input['year']}]"

In [12]:
all_issues = []

epic_dict = {
    'issuetype': {'name': 'Epic'},
    'summary': f"{epic_base_title} {project_input['name']}",
    'description': project_input['description'],
    'project': {'key': project_key},
    'labels': [project_input['sub_domain'], project_input['sub_project']],
    'assignee': {'accountId': team['Fried']},
}

epic_issue = jira.create_issue(fields=epic_dict)
all_issues.append(epic_issue)
print(f"Epic Created: https://{domain}.atlassian.net/browse/{epic_issue.key}")

Epic Created: https://gemeente-amsterdam.atlassian.net/browse/DSO-1591


In [14]:
epic_issue.update(fields={'customfield_10004': 2})

In [15]:
# test feature
feature_dict = {
    'issuetype': {'name': 'Feature'},
    'summary': f"{feature_base_title} {project_input['name']}",
    'description': project_input['description'],
    'project': {'key': project_key},
    'labels': [project_input['sub_domain'], project_input['sub_project']],
    'assignee': {'accountId': team['Fried']},
    'parent': {'key': epic_issue.key},
}

feature_issue = jira.create_issue(fields=feature_dict)
all_issues.append(feature_issue)
print(f"Feature Created: https://{domain}.atlassian.net/browse/{feature_issue.key}")

Feature Created: https://gemeente-amsterdam.atlassian.net/browse/DSO-1592


In [23]:
# test story
story_dict = {
    'issuetype': {'name': 'Story'},
    'summary': f"{story_base_title} {project_input['name']}",
    'description': project_input['description'],
    'project': {'key': project_key},
    'labels': [project_input['sub_domain'], project_input['sub_project']],
    'assignee': {'accountId': team['Fried']},
    'parent': {'key': epic_issue.key},
    'customfield_10004': 2,
}

update_dict = {
    'customfield_10004': 3,
}

story_issue = jira.create_issue(fields=story_dict)
jira.create_issue_link('Relates', story_issue.key, feature_issue.key)
all_issues.append(story_issue)
print(f"Story Created: https://{domain}.atlassian.net/browse/{story_issue.key}")

Story Created: https://gemeente-amsterdam.atlassian.net/browse/DSO-1594


In [22]:
story_issue.update(fields={'customfield_10004': 5})

In [24]:
# delete all issues
for issue in all_issues:
    issue.delete()

In [ ]:
try:
    issue.update(update={'customfield_10004': 2.0})
except Exception as e:
    print(e)